In [1]:
import numpy as np
from fractions import Fraction

In [2]:
x = [1, 2, 4, 8]
f = [0, -2, -1, 2]

# Lagrange interpolation polynomial

In [3]:
def lagrange_polynomial_basis(x, i, q):
    """
    Input:
    x = list of collocation points
    i = 0, 1, ..., n, where n = degree of polynomial and n+1 = numb. of collocation points used for interpolation (len of x)
    q = scalar query point for which function value should be interpolated
    
    Return:
    Basis function for lagrange polynomial
    """
    nominator   = np.prod([(q - x[j]) for j in range(len(x)) if j != i])
    denominator = np.prod([(x[i] - x[j]) for j in range(len(x)) if j != i])
    print("l_",i, "(", q, ") = ", nominator/denominator)
    return nominator/denominator



def lagrange_interpolation(x, f, q):
    """
    Input:
    x = list of collocation points
    f = list function values corresponding to x
    q = scalar query point for which function value should be interpolated
   
    Return:
    Lagrange interpolation at q
    """
    assert (len(x) == len(f))
    
    l = [lagrange_polynomial_basis(x, i, q) for i in range(len(x))] # Get the basis functions
    Ln = sum( [ f[i] * l[i] for i in range(len(x)) ] ) # Weighted sum up the basis functions
    
    print("Ln(", q, ") = ", Ln)
    return Ln

In [4]:
q = 3

In [5]:
# Determine Lagrange polynomial for the above data points and evaluate at q=3
Ln = lagrange_interpolation(x, f, q)

l_ 0 ( 3 ) =  -0.23809523809523808
l_ 1 ( 3 ) =  0.8333333333333334
l_ 2 ( 3 ) =  0.4166666666666667
l_ 3 ( 3 ) =  -0.011904761904761904
Ln( 3 ) =  -2.107142857142857


In [6]:
# Compare with master solution
-5/21, 5/6, 5/12, -1/84, -59/28

(-0.23809523809523808,
 0.8333333333333334,
 0.4166666666666667,
 -0.011904761904761904,
 -2.107142857142857)

# Barycentric formula

In [7]:
def barycentric_weights(x, i):
    """
    Barycentric weights lambda_i
    Independent of query point --> compute only once
    """
    denominator = np.prod([(x[i] - x[j]) for j in range(len(x)) if j != i])
    w = 1/denominator
    return w


def get_mui(x, i, w, q):
    mui = w[i] / (q - x[i])
    return mui


def barycentric_interpolation(x, f, q):
    """
    Input:
    x = list of collocation points
    f = list function values corresponding to x
    q = scalar query point for which function value should be interpolated
    
    Return:
    Barycentric interpolation at q
    """
    assert (len(x) == len(f))
    
    w  = [barycentric_weights(x, i) for i in range(len(x))] # List of barycentric weights for i = 0, 1, .... n 
    print("w = ", w)
    
    mu = [get_mui(x, i, w, q)       for i in range(len(x))] # List of mus for i = 0, 1, .... n
    print("mu = ", mu)
    
    Pn = sum([mu[i] * f[i]          for i in range(len(x))]) / sum(mu)
    print("Pn(", q, ") = ", Pn)
    
    return Pn
    

In [8]:
# Evaluate the interpolation polynomial at x = 3 using the Barycentric formula.
barycentric_interpolation(x, f, q)

w =  [-0.047619047619047616, 0.08333333333333333, -0.041666666666666664, 0.005952380952380952]
mu =  [-0.023809523809523808, 0.08333333333333333, 0.041666666666666664, -0.0011904761904761904]
Pn( 3 ) =  -2.1071428571428568


-2.1071428571428568

In [9]:
# Compare with master solution
-1/21, 1/12, -1/24, 1/168, -59/28

(-0.047619047619047616,
 0.08333333333333333,
 -0.041666666666666664,
 0.005952380952380952,
 -2.107142857142857)

# Aitken-Neville interpolation

In [10]:
def Aitken_Neville_interpolation(x, f, q):
    """
    Implements the Aitken-Neville algorithm:
    Determines interpolation polynomial and evaluates it at a query point q.
    (see BNM script figure 6.2)
    
    Input:
    x = list of collocation points
    f = list function values corresponding to x
    q = scalar query point for which function value should be interpolated
    
    Return:
    Aitken-Neville interpolation at q
    """
    assert (len(x) == len(f))
    
    P = np.zeros((len(x), len(x))) # Array that to store the intermediate results of the Aitken-Neville scheme
    P[0] = f # Initialize first column with function values at collocation points
    
    # Merge columns-wise according to Aitken-Neville Algorithm
    for i in range(1, len(x)):
        for j in range(len(x) - i):
            P[i, j] = P[i-1, j+1] + (q - x[j+i]) / (x[j] - x[j+i]) * (P[i-1, j] - P[i-1, j+1] )
    
    print("Columns of Aitken-Neville scheme:\n", P.T)
    print("Pn(", q, ") = ", P[-1, 0])
    
    return P[-1, 0]

In [11]:
x = [1, 2, 4, 8]
f = [0, -2, -1, 2]
q = 5
Aitken_Neville_interpolation(x, f, q)

Columns of Aitken-Neville scheme:
 [[ 0.         -8.          2.          0.64285714]
 [-2.         -0.5        -0.375       0.        ]
 [-1.         -0.25        0.          0.        ]
 [ 2.          0.          0.          0.        ]]
Pn( 5 ) =  0.6428571428571428


0.6428571428571428

In [12]:
# Compare with master solution
9/14


0.6428571428571429